In [1]:
from flask import Flask, request, jsonify, render_template
import joblib
import io
import base64
from sqlalchemy import create_engine
import pickle



db_user = '***'
db_password = '***'
db_host = '****'
db_port = '****'
db_name = '****'
# Crear la cadena de conexión
churro = f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(churro)


app = Flask(__name__)

@app.route("/", methods=["GET"])
def welcome():
    return "<h1>MPG MODEL PREDICTOR"

# Cargar el modelo, los label encoders y el scaler
kmeans, label_encoders, scaler = joblib.load('clustering_model.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.form
    user_data = pd.DataFrame({
        'rol': [data['rol']],
        'puesto_trabajo': [data['puesto_trabajo']],
        'pais': [data['pais']],
        'interes': [data['interes']],
        'alergenos': [data['alergenos']]
    })
    
    # Transformar las variables categóricas en números
    for column in user_data.columns:
        user_data[column] = label_encoders[column].transform(user_data[column].astype(str))
    
    # Normalizar los datos
    user_data_scaled = scaler.transform(user_data)
    
    # Predecir el cluster
    cluster = kmeans.predict(user_data_scaled)[0]
    
    return jsonify({'cluster': int(cluster)})

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'pymysql'